In [1]:
from z3 import *
import argparse
import itertools
import time
import random

In [2]:
def sum_to_one( ls ):
    # At least one
    atLeastOne = Or(ls)
    atMostOne = True
    for i,j in itertools.combinations(ls,2):
        atMostOne = And(Or(Not(i), Not(j)), atMostOne)
    exactlyOne = And(atLeastOne, atMostOne)
    return exactlyOne

In [3]:
n=8
k=4

In [4]:
vs = [  [Bool ("e_{}_{}".format(i,j))  for j in range(n)] for i in range(k)]

In [5]:
base_cons = []

# add basic constraints
for i in vs:
    base_cons.append(sum_to_one(i))

s = Solver()

s.add( And(base_cons) )

In [6]:
print(vs)

[[e_0_0, e_0_1, e_0_2, e_0_3, e_0_4, e_0_5, e_0_6, e_0_7], [e_1_0, e_1_1, e_1_2, e_1_3, e_1_4, e_1_5, e_1_6, e_1_7], [e_2_0, e_2_1, e_2_2, e_2_3, e_2_4, e_2_5, e_2_6, e_2_7], [e_3_0, e_3_1, e_3_2, e_3_3, e_3_4, e_3_5, e_3_6, e_3_7]]


In [7]:
#print (And(base_cons))
r = s.check()
if r == sat:
    m = s.model()
    for i in vs:
        for j in i:
            if is_true(m[j]):
                print(j)
else:
    print("unsat")

e_0_1
e_1_0
e_2_0
e_3_0


In [8]:
def add_a_guess_solution( guess, reds, whites ):
    guess_cons = True
    redCombs = False
    for rc in itertools.combinations(range(k), reds):
        
        notRC = []
        for i in range(k):
            if i not in rc:
                notRC.append(i)
        
        C1 = True
        for x in rc:
            C1 = And(C1, vs[x][guess[x]])
        
        C2 = False
        for x in notRC:
            C2 = Or(C2, vs[x][guess[x]])
        
        redCombs = Or(redCombs, And(C1, Not(C2)))
    
    others = k - reds - whites
    
    whiteConstraint = False
    for wc in itertools.combinations(range(k), whites):
        
        notWC = []
        for i in range(k):
            if i not in rc:
                notWC.append(i)
        
        temp1 = True
        for x in wc:
            temp = False
            
            for i in range(k):
                if i!=x:
                    temp2 = True
                    for a in notWC:
                        temp2 = And(temp2,Not(vs[a][guess[i]]))
                    temp = Or(temp, And(vs[i][guess[x]], temp2))
            
            temp = And(Not(vs[x][guess[x]]), temp)
            
            temp1 = And(temp1, temp)
        
        whiteConstraint = Or(whiteConstraint, temp1)
    
    guess_cons = And(whiteConstraint, redCombs)
    
    s.add( guess_cons )

In [9]:
color_name =  { 0:'R', 1:'G', 2:'B', 3:'Y', 4:'Br', 5:'O', 6:'Bl', 7:'W', }
if n > 8:
    for i in range(8,n):
        color_name[i] = 'C'+str(i)

In [10]:
def print_move( move ):
    for i in range(k):
        c = color_name[move[i]]
        print(c, end=' '),
    print("\n")

In [11]:
def get_a_solution():
    sol = [0]*k
    if s.check() == sat:
        m = s.model()
        for i in range(k):
            for j in range(n):
                val = m[vs[i][j]]
                if is_true( val ):
                    sol[i] = j
        return sol
    else:
        print("some thing bad happened! no more moves!\n")
        raise Exception('Failed!')

In [12]:
def get_response():
    red = int(input("Enter red count: "))
    white = int(input("Enter white count: "))
    if white+red > k:
        raise Exception("bad input!")
    return red,white


In [13]:
def play_game():
    guess_list = []
    response_list = []
    red = 0
    while red < k:
        if len(guess_list) == 0:
            # TODO: start with random guess
            move = [0,0,1,1]
        else:
            move = get_a_solution()
        guess_list.append(move)
        print("found a move:")
        print_move( move )
        red, white = get_response()
        add_a_guess_solution( move, red, white )
    print("Game solved!")

In [14]:
play_game()

found a move:
R R G G 

Enter red count: 1
Enter white count: 0
found a move:
B G R G 

Enter red count: 1
Enter white count: 1
found a move:
B B G W 

Enter red count: 0
Enter white count: 3
some thing bad happened! no more moves!



Exception: Failed!